# SIIM-COCOtoIce01-Kag


## Environment Setup

In [ ]:
import torch
print(f"The preinstalled torch version is: {torch.__version__}")
version_cuda = torch.version.cuda
print(f"The preinstalled CUDA version is: {torch.version.cuda}")

In [ ]:
!pip install fastai==2.3.1 --upgrade -q

In [ ]:
if version_cuda >= '11.0':
    !pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html --upgrade -q
else:

    !pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html --upgrade -q

!pip install torchtext==0.9.0 --upgrade -q

In [ ]:
!pip install git+git://github.com/airctic/icevision.git --upgrade -q
!pip install git+git://github.com/airctic/icedata.git --upgrade -q

In [ ]:
if version_cuda >= '11.0':
    # Use for CUDA 11.0
    !pip install mmcv-full=="1.3.3" -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html --upgrade -q   
else:
    # Use for CUDA 10.x
    !pip install mmcv-full=="1.3.3" -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.0/index.html --upgrade -q

!pip install mmdet==2.12.0 --upgrade -q
!pip install yolov5-icevision --upgrade -q

In [ ]:
#Restart Kernel    
import IPython
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel
    

## Initialize and Load Dataset

In [ ]:
from icevision.all import *

In [ ]:
import pandas as pd
from datetime import *
import pytz
from icevision.widgets import *

In [ ]:
params = dict(
    size=256,
    size_trans=256,
    batch_size=8
    )

path = Path('./')
extra_args = {}

# Kaggle
path_dicom = Path('../input/siimcovid19coco256px')

path_output_patches = path_dicom / ('DS-COCO_' + str(params['size']))

classlabels = ['Negative', 'Typical', 'Indeterminate', 'Atypical']

In [ ]:
# Parser
parser = parsers.COCOBBoxParser(annotations_filepath=str(path_output_patches) + '/coco_train_annotations_' + str(params['size']) + 'px.json', img_dir=path_output_patches / 'train')

In [ ]:
# Parsing
train_rs, valid_rs = parser.parse()

In [ ]:
class_map = ClassMap(classlabels)
print(len(class_map))
class_map

In [ ]:
show_records(train_rs[:6], ncols=3, class_map=class_map)

In [ ]:
# Transforms
train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=params['size'], presize=params['size_trans']), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(params['size']), tfms.A.Normalize()])

In [ ]:
# Datasets
train_ds = Dataset(train_rs, train_tfms)
valid_ds = Dataset(valid_rs, valid_tfms)

### Model Selection

In [ ]:
# Just change the value of selection to try another model

selection = 0

extra_args = {}

if selection == 0:
  model_type = models.mmdet.retinanet
  backbone = model_type.backbones.resnet50_fpn_1x

elif selection == 1:
  # The Retinanet model is also implemented in the torchvision library
  model_type = models.torchvision.retinanet
  backbone = model_type.backbones.resnet50_fpn

elif selection == 2:
  model_type = models.ross.efficientdet
#   backbone = model_type.backbones.tf_lite0
  backbone = model_type.backbones.tf_d0
  # The efficientdet model requires an img_size parameter
  extra_args['img_size'] = params['size']

elif selection == 3:
  model_type = models.ultralytics.yolov5
  backbone = model_type.backbones.small
  # The yolov5 model requires an img_size parameter
  extra_args['img_size'] = params['size']

model_type, backbone, extra_args

In [ ]:
# Instantiate the mdoel
model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(class_map), **extra_args) 

In [ ]:
# DataLoaders
train_dl = model_type.train_dl(train_ds, batch_size=params['batch_size'], num_workers=0, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=params['batch_size'], num_workers=0, shuffle=False)

In [ ]:
# model_type.show_batch(first(valid_dl), ncols=4)

## Model & Parameter Setup

In [ ]:
params['run_datetime'] = datetime.now(pytz.timezone('US/Pacific')).strftime("%m%d%Y-%H%M")


model_save_name = 'ModelName' + '_'  
model_save_name += str(len(parser.class_map)) + 'cls_' 
model_save_name += str((params['size'])) + 'px_'
model_save_name += str(len(train_ds)) + 'ds_'
model_save_name += params['run_datetime']
print(model_save_name)

In [ ]:
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

In [ ]:
learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

In [ ]:
# learn.freeze()
learn.lr_find()

## Model Training - Round 1

In [ ]:
params.update(lr1 = 1e-3)
params.update (epochs1 = 3)

learn.freeze_to(-2)
learn.fit_one_cycle(params['epochs1'], params['lr1'])

In [ ]:
learn.unfreeze()

params.update(lr1 = 1e-3)
params.update (epochs1 = 15)

learn.fit(params['epochs1'], params['lr1'])

In [ ]:
params.update(current_saved_model='stage-1-' + model_save_name)
params['current_saved_model']
learn.save(path/ params['current_saved_model'])

## Review Model Run - Validation Images

In [ ]:
model_type.show_results(model, valid_ds, detection_threshold=.3)